In [1]:
import jax
import jax.numpy as jnp
import jax.random as jrandom
import src.data.ToyData as ToyData
import src.models as models
import src.Trainer as Trainer
import src.SDESolver as SDESolver
import src.SDE as SDE




In [2]:
data_generator = ToyData.CircleDataGenerator(landmark_num=10, radius=1, center=jnp.array([0, 0]))
trainer = Trainer.SsmTrainer()
model = models.DsmModel(dim=2, hidden_dims=(128, 256, 128), with_x0=True)
brownian_sde = SDE.Brownian_Motion_SDE(dim=2, sigma=1)
x_0 = data_generator.generate_data(jrandom.PRNGKey(0), 1000)
sde_solver = SDESolver.EulerMaruyama.from_sde(brownian_sde, 0.01, 1, 10, 2, jrandom.PRNGKey(0))
xs, _ = jax.vmap(sde_solver.solve, in_axes=(0, None))(x_0, jrandom.PRNGKey(0))
print(xs.shape)
print(x_0.shape)
train_state = trainer.train_state_init(jrandom.PRNGKey(0), model, lr=1e-3, model_kwargs={'x': x_0[0], 't': jnp.array([0]), 'x0': x_0[0]})
train_state, losses = trainer.train(train_state, brownian_sde, sde_solver, data_generator, 100, 1000)


(1000, 101, 10, 2)
(1000, 10, 2)


TypeError: mul got incompatible shapes for broadcasting: (64,), (99,).